## Notebook from machine-learning-with-ontologies

In [1]:
import click as ck
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.framework import function
import re
import math
import matplotlib.pyplot as plt
import logging
from tensorflow.keras.layers import Input
from tensorflow.keras import optimizers
from tensorflow.keras import constraints
#from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from tensorflow.keras import backend as K
from scipy.stats import rankdata
import os

from elembeddings.elembedding import (
    ELModel, load_data, load_valid_data, Generator, MyModelCheckpoint)

2022-06-10 13:01:08.228438: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-10 13:01:08.228454: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Set parameters

In [2]:
# Parameters
batch_size = 256
embedding_size = 50
margin = -0.1
reg_norm = 1
learning_rate = 1e-3
epochs = 20
org_id = '9606'

### Load training and validation data

In [3]:
# Load training data in (h, l, t) triples
# classes and relations are entity to id mappings
train_data, classes, relations = load_data(f'data/train/{org_id}.classes-normalized.owl')
valid_data = load_valid_data(f'data/valid/{org_id}.protein.links.v11.0.txt', classes, relations)
"""for key, value in train_data.items():
    print(f'{key} {len(value)}')
for key, value in classes.items():
    print(f'{key}: {value}')
for key, value in relations.items():
    print(f'{key}: {value}')
for h, l, t in valid_data:
    print(f'{h}: {l}: {t}')"""

"for key, value in train_data.items():\n    print(f'{key} {len(value)}')\nfor key, value in classes.items():\n    print(f'{key}: {value}')\nfor key, value in relations.items():\n    print(f'{key}: {value}')\nfor h, l, t in valid_data:\n    print(f'{h}: {l}: {t}')"

In [4]:
# Filter out protein classes
proteins = {}
for k, v in classes.items():
    if not k.startswith('<http://purl.obolibrary.org/obo/GO_'):
        proteins[k] = v

# Prepare data for training the model
nb_classes = len(classes)
nb_relations = len(relations)
nb_train_data = 0
for key, val in train_data.items():
    nb_train_data = max(len(val), nb_train_data)
train_steps = int(math.ceil(nb_train_data / (1.0 * batch_size)))
train_generator = Generator(train_data, batch_size, steps=train_steps)

# id to entity maps
cls_dict = {v: k for k, v in classes.items()}
rel_dict = {v: k for k, v in relations.items()}

cls_list = []
rel_list = []
for i in range(nb_classes):
    cls_list.append(cls_dict[i])
for i in range(nb_relations):
    rel_list.append(rel_dict[i])

        
print('Total number of classes', nb_classes)
print('Total number of relations', nb_relations)

Total number of classes 61810
Total number of relations 10


### Build ELEmbeddings Model and Train

Embeddings are saved depending on mean rank evaluation on validation set

In [5]:
# Input layers for each loss type
nf1 = Input(shape=(2,), dtype=np.int32)
nf2 = Input(shape=(3,), dtype=np.int32)
nf3 = Input(shape=(3,), dtype=np.int32)
nf4 = Input(shape=(3,), dtype=np.int32)
dis = Input(shape=(3,), dtype=np.int32)
top = Input(shape=(1,), dtype=np.int32)
nf3_neg = Input(shape=(3,), dtype=np.int32)

# Build model
el_model = ELModel(nb_classes, nb_relations, embedding_size, batch_size, margin, reg_norm)
out = el_model([nf1, nf2, nf3, nf4, dis, top, nf3_neg])
model = tf.keras.Model(inputs=[nf1, nf2, nf3, nf4, dis, top, nf3_neg], outputs=out)
optimizer = optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mse')

# Pandas files to store embeddings
if not os.path.exists('data/elembeddings'):
    os.makedirs('data/elembeddings')
out_classes_file = f'data/elembeddings/{org_id}_cls_embeddings.pkl'
out_relations_file = f'data/elembeddings/{org_id}_rel_embeddings.pkl'

# ModelCheckpoint which runs at the end of each epoch
checkpointer = MyModelCheckpoint(
    save_freq='epoch',
    out_classes_file=out_classes_file,
    out_relations_file=out_relations_file,
    cls_list=cls_list,
    rel_list=rel_list,
    valid_data=valid_data,
    proteins=proteins,
    monitor='loss')

# Start training
model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=epochs,
    workers=12,
    callbacks=[checkpointer,])


2022-06-10 13:01:25.243367: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-10 13:01:25.243839: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-10 13:01:25.243951: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-06-10 13:01:25.244043: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-06-10 13:01:25.244136: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

hello2
Epoch 1/20
4206/4206 [==============================] - ETA: 0s - loss: 14.2395
 Validation 1 4318.369714409735


 Saving embeddings 1 4318.369714409735

4206/4206 [==============================] - 639s 151ms/step - loss: 14.2395
Epoch 2/20
4205/4206 [============================>.] - ETA: 0s - loss: 1.3521
 Validation 2 1866.987419359667


 Saving embeddings 2 1866.987419359667

4206/4206 [==============================] - 624s 148ms/step - loss: 1.3519
Epoch 3/20
4206/4206 [==============================] - ETA: 0s - loss: 0.2944
 Validation 3 2674.5984710143844

4206/4206 [==============================] - 617s 147ms/step - loss: 0.2944
Epoch 4/20
4206/4206 [==============================] - ETA: 0s - loss: 0.1200
 Validation 4 4251.966587585505

4206/4206 [==============================] - 612s 146ms/step - loss: 0.1200
Epoch 5/20
4206/4206 [==============================] - ETA: 0s - loss: 0.0815
 Validation 5 5054.082646050681

4206/4206 [==============================] -

### Evaluation of embeddings on the test set

In [6]:
def load_test_data(data_file, classes, relations):
    data = []
    rel = f'<http://interacts>'
    with open(data_file, 'r') as f:
        for line in f:
            it = line.strip().split()
            id1 = f'<http://{it[0]}>'
            id2 = f'<http://{it[1]}>'
            if id1 not in classes or id2 not in classes or rel not in relations:
                continue
            data.append((id1, rel, id2))
    return data

def compute_rank_roc(ranks, n_prots):
    auc_x = list(ranks.keys())
    auc_x.sort()
    auc_y = []
    tpr = 0
    sum_rank = sum(ranks.values())
    for x in auc_x:
        tpr += ranks[x]
        auc_y.append(tpr / sum_rank)
    auc_x.append(n_prots)
    auc_y.append(1)
    auc = np.trapz(auc_y, auc_x) / n_prots
    return auc


# Pandas files to store embeddings
out_classes_file = f'data/elembeddings/{org_id}_cls_embeddings.pkl'
out_relations_file = f'data/elembeddings/{org_id}_rel_embeddings.pkl'

cls_df = pd.read_pickle(out_classes_file)
rel_df = pd.read_pickle(out_relations_file)
nb_classes = len(cls_df)
nb_relations = len(rel_df)
embeds_list = cls_df['embeddings'].values
rembeds_list = rel_df['embeddings'].values
size = len(embeds_list[0])
embeds = np.zeros((nb_classes, size), dtype=np.float32)
for i, emb in enumerate(embeds_list):
    embeds[i, :] = emb

rs = np.abs(embeds[:, -1]).reshape(-1, 1)
embeds = embeds[:, :-1]
prot_index = list(proteins.values())
prot_rs = rs[prot_index, :]
prot_embeds = embeds[prot_index, :]
prot_dict = {v: k for k, v in enumerate(prot_index)}
    
rsize = len(rembeds_list[0])
rembeds = np.zeros((nb_relations, rsize), dtype=np.float32)
for i, emb in enumerate(rembeds_list):
    rembeds[i, :] = emb

train_data = load_test_data(f'data/train/{org_id}.protein.links.v11.0.txt', classes, relations)
valid_data = load_test_data(f'data/valid/{org_id}.protein.links.v11.0.txt', classes, relations)
trlabels = {}
for c, r, d in train_data:
    c, r, d = prot_dict[classes[c]], relations[r], prot_dict[classes[d]]
    if r not in trlabels:
        trlabels[r] = np.ones((len(prot_embeds), len(prot_embeds)), dtype=np.int32)
    trlabels[r][c, d] = 1000
for c, r, d in valid_data:
    c, r, d = prot_dict[classes[c]], relations[r], prot_dict[classes[d]]
    if r not in trlabels:
        trlabels[r] = np.ones((len(prot_embeds), len(prot_embeds)), dtype=np.int32)
    trlabels[r][c, d] = 1000

test_data = load_test_data(f'data/test/{org_id}.protein.links.v11.0.txt', classes, relations)
top1 = 0
top10 = 0
top100 = 0
mean_rank = 0
ftop1 = 0
ftop10 = 0
ftop100 = 0
fmean_rank = 0
labels = {}
preds = {}
ranks = {}
franks = {}
eval_data = test_data
n = len(eval_data)
for c, r, d in eval_data:
    c, r, d = prot_dict[classes[c]], relations[r], prot_dict[classes[d]]
    if r not in labels:
        labels[r] = np.zeros((len(prot_embeds), len(prot_embeds)), dtype=np.int32)
    if r not in preds:
        preds[r] = np.zeros((len(prot_embeds), len(prot_embeds)), dtype=np.float32)
    labels[r][c, d] = 1
    ec = prot_embeds[c, :]
    rc = prot_rs[c, :]
    er = rembeds[r, :]
    ec += er

    # Compute similarity
    dst = np.linalg.norm(prot_embeds - ec.reshape(1, -1), axis=1)
    dst = dst.reshape(-1, 1)
    res = np.maximum(0, dst - rc - prot_rs - margin)
    res = res.flatten()

    preds[r][c, :] = res
    index = rankdata(res, method='average')
    rank = index[d]
    if rank == 1:
        top1 += 1
    if rank <= 10:
        top10 += 1
    if rank <= 100:
        top100 += 1
    mean_rank += rank
    if rank not in ranks:
        ranks[rank] = 0
    ranks[rank] += 1

    # Filtered rank
    index = rankdata((res * trlabels[r][c, :]), method='average')
    rank = index[d]
    if rank == 1:
        ftop1 += 1
    if rank <= 10:
        ftop10 += 1
    if rank <= 100:
        ftop100 += 1
    fmean_rank += rank

    if rank not in franks:
        franks[rank] = 0
    franks[rank] += 1
top1 /= n
top10 /= n
top100 /= n
mean_rank /= n
ftop1 /= n
ftop10 /= n
ftop100 /= n
fmean_rank /= n

rank_auc = compute_rank_roc(ranks, len(proteins))
frank_auc = compute_rank_roc(franks, len(proteins))

print(f'Evaluation for {org_id}')
print(f'{top10:.2f} {top100:.2f} {mean_rank:.2f} {rank_auc:.2f}')
print(f'{ftop10:.2f} {ftop100:.2f} {fmean_rank:.2f} {frank_auc:.2f}')

Evaluation for 9606
0.01 0.14 1800.38 0.89
0.01 0.15 1766.14 0.89
